# **Acetaminophen Attenuates Perception of Auditory Stimuli**
# ***Demographics***

In [24]:
# basic coding/ML tools
import numpy as np, pandas as pd, os, sys, scipy, xlrd, urllib, pandas_profiling, itertools, re, warnings
from jupyter_contrib_nbextensions.application import main
from ipywidgets import interact
from pandas.api.types import CategoricalDtype
from scipy import stats
from scipy.stats import ttest_ind
from fancyimpute import KNN
from pandas_profiling import ProfileReport

# ML tools
import sklearn, statsmodels.api as sm, statsmodels.formula.api as smf, cvxopt, pingouin as pg
from statsmodels.stats.anova import AnovaRM
from xgboost import XGBRegressor, XGBClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import preprocessing, metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.dummy import DummyRegressor
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_squared_log_error, make_scorer, confusion_matrix, plot_confusion_matrix, classification_report, multilabel_confusion_matrix
from imblearn.over_sampling import SMOTE

# visualization tools
import matplotlib.pyplot as plt, qgrid, seaborn as sns, IPython.display
from statsmodels.graphics.factorplots import interaction_plot
from IPython.display import Image, HTML
from plotnine import *
%matplotlib inline
plt.rcParams['figure.figsize'] = (14,4)

# View tab (Taken from Martin: https://stackoverflow.com/users/2575273/martin)
def View(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))
files = os.listdir(os.curdir)

# set directory
os.chdir("/Users/home/Desktop/Research/Tylenol/Analysis")
print(os.getcwd())

# only get warnings once
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# # toggle on/off code!
# HTML('''<script>
# code_show=true; 
# function code_toggle() {
#  if (code_show){
#  $('div.input').hide();
#  } else {
#  $('div.input').show();
#  }
#  code_show = !code_show
# } 
# $( document ).ready(code_toggle);
# </script>
# <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

/Users/home/Desktop/Research/Tylenol/Analysis


In [18]:
wide = pd.read_csv('wide_data_final.csv')

#demographics = wide.loc[:,'FixedID':'Gender']
demographics.head(2)

,FixedID,DrugPlacebo,MedsEffectiveness,LastTimeTookMeds,FrequencyTakeMeds,Politics,ChildSES,AdultSES,Height,Weight,SleepHours,WhenLastAte,HowMuchLastEat,SleepQuality,IllnessSeverity,exerciseMinToday,ExerciseRegularMins,LastConsumeCaffeineHours,GeneralHealth,SubjectiveIllness,WhenLastSick,NumDoctorVisits,TakeMedsRecentlyYN,BirthControlYN,MarijuanaFrequency,AlcoholAvg,ArthritisYN,Age,YearUniversity,NeedToBelong,Nostalgia,EarlyFamilyEnvironment,PosPANAS,NegPANAS,Mellow,Unpretentious,Sophisticated,Intense,Contemporary,PT,FS,EC,PD,E,A,C,N,O,AIMS,Race_Asian,Race_Black,Race_Hispanic,Race_Other,Race_White,PoliticalParty,PreferredMeds,Gender
0,1.0,0,4.0,1095.0,1.0,3.0,4.666667,4.000000,73.0,163.0,5.0,0.666667,2.0,5.0,1.000,0.0,360.0,0.667,3.0,1.666667,2.0,0.0,0.0,0.0,1.0,0.0,0.0,21.0,3.0,29.0,3.857143,2.153846,25.0,17.0,5.0,4.000000,4.285714,4.25,4.25,2.142857,2.142857,1.857143,2.000000,3.00,3.111111,3.444444,2.750,3.0,98.0,1,0,0,0,0,other,Other,male
1,2.0,1,4.0,365.0,4.0,5.0,6.000000,5.333333,69.0,168.0,8.0,3.000000,3.0,9.0,1.375,15.0,300.0,5.000,4.0,1.333333,4.0,0.0,1.0,0.0,1.0,1.0,0.0,19.0,2.0,24.0,4.142857,2.923077,35.0,14.0,5.0,3.666667,3.428571,3.25,5.25,3.142857,1.857143,2.428571,1.285714,2.75,3.777778,3.111111,1.375,3.9,114.0,0,0,0,0,1,libertarian,Other,male


In [20]:
demographic_report = ProfileReport(demographics)
demographic_report.to_file("acetaminophen demographics.html")